In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
import os
os.chdir('../')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import tcav
from src.download_data import download_imagenet, download_random_imagenet_classes
import pickle
from src.dataset import *
from src.concept_vectors import *
from src.util import *
import random
from src.plots import *
from sklearn.manifold import TSNE
from scipy.spatial.distance import cosine 
import pandas as pd
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from sklearn.datasets import make_blobs
from src.hierarchy import *

2023-01-04 23:08:13.781666: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
load_cem_vectors("xor",0,42)

array([[-2.82513979e-03,  1.83242887e-01, -2.54715537e-03,
        -6.62477873e-03, -6.82961021e-04,  2.72987664e-01,
        -5.89384884e-03, -5.05319284e-03, -2.17477768e-03,
        -4.35372209e-03,  2.29641631e-01,  2.51608282e-01,
        -8.27206066e-04,  1.24053180e-01, -1.18229364e-03,
        -8.37748777e-03],
       [-3.96880694e-03,  7.61961341e-02, -2.26540561e-03,
        -8.55666958e-03, -1.25119681e-04,  2.51613081e-01,
        -4.38643992e-03, -3.62144527e-03, -3.24593834e-03,
        -3.40218539e-03,  1.27558887e-01,  2.28207290e-01,
        -2.75836879e-04,  3.56483400e-01, -1.78769080e-03,
        -9.20734461e-03],
       [-3.82020924e-04,  3.45478117e-01, -3.30812228e-03,
        -3.02043254e-03, -2.11795466e-03,  2.55956262e-01,
        -7.93734379e-03, -6.98391581e-03, -4.19573917e-04,
        -5.93684847e-03,  3.78275573e-01,  3.26836884e-01,
        -2.01741024e-03, -3.26105556e-03, -1.82194257e-04,
        -6.40609954e-03],
       [ 8.66556019e-02,  2.98988849e